<font size=6  color= 'white' > <b>[LEPL1507] - PROJET P4</b> <br>
<br><br>
Airson Alexis<br>
Beniffou Ibrahim <br>
Henneaux Lucas <br>
Lemaire Antoine <br>
Smith Marielle <br>
Canon Théo<br>
<div style="text-align: right"> </div>

<br><br>
</font>


This is the version maintaining the essential parts of code to run

Easier to add new features 


<font size=6  color= 'white' > <b> Phase 1 : retrieving data </b> <br>

The aim of this first part is to get arrays of images 

In [ ]:
# CELL 1.1 : accessing to data 
# Needs [ CELL 0 : import ]

# defines variables 

#-----------------------------------------------------------------------------

path_training = 'BelgiumTSC_Training\Training'
path_testing = 'BelgiumTSC_Testing\Testing'

nbr_class = 62 # len(next(walk(path_training))[1])  counting the number of classes 
print('number of classes : ', nbr_class)

#-----------------------------------------------------------------------------

# resol is the format for the images that we want for convenience

resol = (50,50) # Square image only ! 
print("Resolution of images : ", resol)

In [ ]:
from retrieving_data import accessing_2

# These are iterators 
# See fct accessing in codes of Phase 1 
training_set = accessing_2(path_training)
test_set = accessing_2(path_testing) 

In [ ]:
from retrieving_data import store_2

# These are arrays
# See fct store in codes of Phase 1 
X_train, y_train = store_2(training_set, [],[], resol)
X_test, y_test = store_2(test_set, [], [], resol)

<font size=6  color= 'white' > <b> Phase 2 : visualisation of the dataset </b> <br>

The aim of this part is to visualise the initial datasets 

In [ ]:
from Visu import number
from copy import deepcopy

initial_nbr_train = number(nbr_class, deepcopy(y_train))
initial_nbr_test = number(nbr_class, deepcopy(y_test))

In [ ]:
from Visu import graphs
from copy import deepcopy

graphs(nbr_class, deepcopy(y_train), deepcopy(y_test), 'y_train', 'y_test')

<font size=6  color= 'white' > <b> Phase 3 : randomization of the dataset </b> <br>

The aim of this part is to avoid overfitting by applying transformations on the images 

In [ ]:
from keras import Sequential
from keras.layers import RandomZoom, RandomRotation, RandomCrop, RandomContrast
from tensorflow import get_logger
from copy import deepcopy
from numpy import array

# This cell is about data augmentation 
# We'll randomize our set and do our augmentation with this (cells later)
# We choose to work with the second method of augmentation  

#-----------------------------------------------------------------------------

get_logger().setLevel('ERROR')

data_augmentation = Sequential() 

data_augmentation.add(RandomZoom(height_factor=(-0.3, -0.2),width_factor=(-0.3, -0.2)))
data_augmentation.add(RandomRotation((-0.05, 0.05)))
data_augmentation.add(RandomCrop(height=resol[0],width=resol[1]))
data_augmentation.add(RandomContrast((1.0,2.0)))

#-----------------------------------------------------------------------------

# We apply the augmentation on our datasets 
augmented_image_train = array(data_augmentation(deepcopy(X_train)))

#-----------------------------------------------------------------------------

print("Shape of the randomized training set : ", augmented_image_train.shape)

In [ ]:
from scipy.ndimage import gaussian_filter
augmented_image_train = gaussian_filter(augmented_image_train, 0.3)

In [ ]:
from random import randint 
from matplotlib.pyplot import figure, imshow
from numpy import zeros 

# A little test to see the result of the augmentation 

#-----------------------------------------------------------------------------

index = randint(0,len(X_train))

visu_augmented = zeros((50,50,3))

for i in range (3) : 
    for j in range (resol[0]) : 
        for k in range(resol[1]) : 
            visu_augmented [j,k,i] = augmented_image_train[index][j,k,i] / 255

figure()
imshow(X_train[index])
figure()
imshow(visu_augmented)

<font size=6  color= 'white' > <b> Phase 4 : augmentation of the dataset </b> <br>

The aim of this part is to change the sizes of the initial datasets


In [ ]:
from aug import first 
from copy import deepcopy

X_train_first, y_train_first = first(nbr_class, deepcopy(initial_nbr_train), deepcopy(initial_nbr_test), deepcopy(augmented_image_train), deepcopy(y_train), resol)

In [ ]:
from Visu import graphs, number

nbr_train_first = number(nbr_class, y_train_first)
graphs(nbr_class, y_train_first.copy(), y_test.copy(), 'y_train_first', 'y_test')

In [ ]:
from aug import second 
from copy import deepcopy

X_train_second, y_train_second = second(nbr_class, deepcopy(initial_nbr_train), deepcopy(augmented_image_train), deepcopy(y_train), resol, max(initial_nbr_train))

In [ ]:
from Visu import graphs, number

nbr_train_second = number(nbr_class, y_train_second)
graphs(nbr_class, y_train_second.copy(), y_test.copy(), 'y_train_second', 'y_test')

In [ ]:
from keras.utils import to_categorical
from copy import deepcopy

# We put our results to categorical 

#-----------------------------------------------------------------------------

y_test_tc = to_categorical(deepcopy(y_test), nbr_class)
y_train_tc = to_categorical(deepcopy(y_train), nbr_class)

print("Shape of y_train without augmentation : ", y_train_tc.shape)
print("Shape of y_test without augmentation : ", y_test_tc.shape)

#-----------------------------------------------------------------------------

y_train_first_tc = to_categorical(y_train_first, nbr_class)


print("Shape of y_train with first method : ", y_train_first_tc.shape)
print("Shape of y_test with first method : ", y_test_tc.shape)

#-----------------------------------------------------------------------------

y_train_second_tc = to_categorical(y_train_second, nbr_class)

print("Shape of y_train with second method : ", y_train_second_tc.shape)
print("Shape of y_test with second method : ", y_test_tc.shape)

<font size=6  color= 'white' > <b> Phase 5 : Construction of the model </b> <br>

The aim of this part is to construct a model using CNN

You can find an optimizer for hyperparameters with the codes of this part

In [ ]:
from Model.model import construct_model
from copy import deepcopy

model_simple = construct_model(nbr_class, deepcopy(X_train))

model_first = construct_model(nbr_class, deepcopy(X_train_first))

model_second = construct_model(nbr_class, deepcopy(X_train_second))

model_second.summary() # Same model so same parameters for the 3 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from copy import deepcopy

# We train here the model with the first method of augmentation 

eps = 3 # The number of time we want the model to train on the entire training_set

#-----------------------------------------------------------------------------

mycallbacks = [EarlyStopping(monitor='val_loss', patience=5)]

#-----------------------------------------------------------------------------

validation_X = deepcopy(X_test)
validation_y = deepcopy(y_test_tc)

#-----------------------------------------------------------------------------

anc_second = model_second.fit(X_train_second, y_train_second_tc,validation_data=(validation_X,validation_y), epochs=eps, callbacks=mycallbacks)

#-----------------------------------------------------------------------------

#added aug.flow to (X_train, y_train) to do data augmentation
# validation_data=(X_test_new_first, y_test_new_first)
# validation_data=(X_test, y_test)

# talk : aug.flow ? 
# talk : validation data strange 

<font size=6  color= 'white' > <b> Phase 6 : Analysis of performances </b> <br>

The aim of this part is to analyse the results of the model based on the testing set available 

In [ ]:
label_names = open("dict.csv").read().strip().split("\n")[0:]
label_names = [l.split(",")[1] for l in label_names]

In [ ]:
from perf import perf 

perf(anc_second)

In [ ]:
from perf import ratio_kaggle

ratio_kaggle(y_test_tc, X_test, model_second)

In [ ]:
from perf import network

from warnings import filterwarnings

filterwarnings('ignore')

network(model_second, y_test_tc, X_test, label_names, nbr_class)

<font size=6  color= 'white' > <b> Phase 7 : Visualisation of the first kaggle challenge </b> <br>

The aim of this part is to visualise the dataset of the first challenge 

In [ ]:
from Visu import to_jpeg

folder_dir = "challenge_1/eval_kaggle1"
to_jpeg(folder_dir)

<font size=6  color= 'white' > <b> Phase 8 : Generating csv for kaggle </b> <br>

The aim of this part is to predict on the kaggle dataset and generate a csv 

In [ ]:
from CSV import store_2

images, names = store_2('challenge_1/eval_kaggle1', resol)

In [ ]:
from csv import DictReader

# We will store the data in dict.csv in a dict 

data = {}

with open('dict.csv', 'r') as f:
    d_reader = DictReader(f, fieldnames=["num", "sign"])

    #get fieldnames from DictReader object and store in list
    for row in d_reader:
        data[row['num']] = row['sign']

In [ ]:
from matplotlib.pyplot import figure, title, imshow 
from numpy import uint8

# Here's a little test to visualise some results 
# We then print the image with num and the sign predicted as a title 

predictions = model_second.predict(images).argmax(axis=1) 

visu_pred = zeros(images.shape)

for i in range (images.shape[0]):
    for j in range (images.shape[3]) : 
        for k in range (images.shape[1]):
            for l in range (images.shape[2]): 
                visu_pred[i,k,l,j] = images[i,k,l,j] / 255

for i in range(20): 
    figure(figsize = (10,10))
    imshow(visu_pred[i])
    sign = data[str(predictions[i])]
    title(str(predictions[i]) + " : " + str(sign))



In [ ]:
# We generate the csv file for kaggle 

from CSV import write 

write(names, predictions, 'kaggle_1')

<font size=6  color= 'white' > <b> Part 2 : Cropping </b> <br>

<font size=6  color= 'white' > <b> Phase 1 : Detecting forms </b> <br>

The aim of this part is to detect forms and contours in the image

In [ ]:
from cv2 import imread, bitwise_and, isContourConvex, boundingRect, minEnclosingCircle, contourArea, imwrite, rectangle, drawContours, waitKey, cvtColor, COLOR_BGR2RGB
from os.path import basename, realpath
from numpy import pi 

from contours import edges, cont, HSV, parameters

def process_shape(raw_image, dest, f) : 

    process_image = raw_image.copy() #copy the image

#----------------------------------------------------------------------------------------------------------

    edge_detected_image = edges(process_image)

    contours = cont(edge_detected_image)

#----------------------------------------------------------------------------------------------------------

    process_image = raw_image.copy()

    mask = HSV(process_image)

    # Apply mask to image to remove grayscale regions
    masked_image = bitwise_and(process_image, process_image, mask=mask)

#----------------------------------------------------------------------------------------------------------

    edge_detected_image = edges(masked_image)

    contours = cont(edge_detected_image)

#----------------------------------------------------------------------------------------------------------

    shapes = [] 
    for contour in contours:

        length, approx, area, perimeter, circularity = parameters(contour)

        #----------------------------------------------------------------------------------------------------------
        # A lot of conditions to classify forms 
        
        if circularity < 0.8:
            continue

        if not isContourConvex(approx):
            continue

        if len(approx) == 3: # We're looking for triangles 
            shapes.append(contour)

        elif len(approx) == 4: # We're looking for squares and losanges 
            (x, y, w, h) = boundingRect(approx)
            ar = w / float(h)
            if (ar >= 0.8 and ar <= 1.2):
                shapes.append(contour)
            else:
                continue

        elif len(approx) == 8: # We're looking for octogones
            shapes.append(contour)

        elif len(approx) > 8: # check if contour has a circular shape
            (x, y), radius = minEnclosingCircle(contour)
            area = contourArea(contour)
            if radius > 0 and area / (pi * radius**2) >= 0.8:
                shapes.append(contour)

        else:
            continue

    #----------------------------------------------------------------------------------------------------------
    # Find contours with largest areas for each shape

    try:
        best_contour = max(shapes, key=lambda x: contourArea(x))
    except ValueError:
        best_contour = None

    if best_contour is None:
        print(' No shape found')
        #bounding box is the whole image
        imwrite(dest + "/" + basename(realpath(f)).replace('.ppm', '.jpeg'), raw_image)
        
        return False
    
    #----------------------------------------------------------------------------------------------------------
    # If it worked, crop the image with a rectangle around the detected shape 
    
    (x,y,w,h) = boundingRect(best_contour)
    rectangle(raw_image, (x,y), (x+w,y+h), (0,255,0), 2)
    drawContours(raw_image, [best_contour],  -1, (255,0,0), 2)
    waitKey()

    raw_image = raw_image[y:y + h, x:x + w]
    img = imread(f)
    img = cvtColor(img,COLOR_BGR2RGB)   

    imwrite(dest + "/"+ basename(realpath(f)).replace('.ppm', '.jpeg'), raw_image)

    return True

In [ ]:
from cv2 import imread, GaussianBlur, imshow, waitKey, medianBlur, drawContours, COLOR_BGR2HSV, inRange, bitwise_or, findContours, RETR_EXTERNAL, CHAIN_APPROX_SIMPLE, arcLength, contourArea, convexHull
from numpy import array, pi 
from os.path import basename, realpath

from retrieving_data import accessing_2

def process(path, dest):

    #----------------------------------------------------------------------------------------------------------

    names = accessing_2(path)

    #----------------------------------------------------------------------------------------------------------

    for f in names : 

        raw_image = imread(f) #load the image
        process_image = raw_image.copy() #copy the image
        if(process_shape(raw_image, dest, f)):
            continue

        # On floute avec du gaussian puis du median blur
        blur = GaussianBlur(process_image, (9, 9), 0)
        blur = medianBlur(blur, 9)

        # On transforme l'image en image HSV
        img_hsv = cvtColor(blur, COLOR_BGR2HSV)
        img_hsv[...,1] = img_hsv[...,1]*1.4


        # Définit la range du rouge en HSV
        lower_red = array([0, 50, 50])
        upper_red = array([10, 255, 255])

        # Crée un masque pour la couleur rouge en utilisant la range définie
        mask1 = inRange(img_hsv, lower_red, upper_red)
        # Masque rouge valeurs hautes
        lower_red = array([170, 70, 50])
        upper_red = array([180, 255, 255])
        mask_red = inRange(img_hsv.copy(), lower_red, upper_red)

        mask_red = bitwise_or(mask1, mask_red)

#----------------------------------------------------------------------------------------------------------

        # Masque bleu
        lower_blue = array([90, 100, 100])
        upper_blue = array([121, 255, 255])
        mask_blue = inRange(img_hsv.copy(), lower_blue, upper_blue)

        #Total mask
        mask_red_blue = bitwise_or(mask_red, mask_blue)

        #show image with mask
        # imshow('mask', mask_red_blue)
        # waitKey()

#----------------------------------------------------------------------------------------------------------

        # Trouve les contours dans l'image filtrée
        contours, _ = findContours(mask_red_blue, RETR_EXTERNAL, CHAIN_APPROX_SIMPLE)

        #draw image with contours
        drawContours(process_image, contours, -1, (0, 255, 0), 3)
        # imshow('contour', process_image)
        # waitKey()

        # Initialise une liste vide pour stocker les contours valides
        list_contour = []
        best_contour = None

        # Parcourt tous les contours trouvés
        for cnt in contours:

            # Calcule le rectangle englobant le contour
            x, y, w, h = boundingRect(cnt)
            #imposer que le bounding box soit plus grand que 5% de l'image 
            
            if (w*h) < (0.0005*raw_image.shape[0]*raw_image.shape[1]):
                print("condition 1")
                continue

            #imposer que le bounding box soit pas un rectangle très plat 
            if (w/h) < 0.7 or (w/h) > 1.3:
                print("condition 2")
                continue

            #if it is a circle very likely to be a round road sign
            area = contourArea(cnt)
            perimeter = arcLength(cnt, True)
            circularity = 4 * pi * area / perimeter ** 2
            if circularity > 0.8:
                best_contour = cnt
                list_contour.append(cnt)
                break
            
            #convexité ? 
            hull = convexHull(cnt)
            hull_area = contourArea(hull)
            solidity = float(area)/hull_area
            if solidity < 0.7:
                print("condition 3")
                continue
            
            list_contour.append(cnt)

        print(len(list_contour))

        after_contour = raw_image.copy()
        drawContours(after_contour, list_contour, -1, (0, 255, 0), 3)
        # imshow('after contour', after_contour)
        # waitKey()

        #----------------------------------------------------------------------------------------------------------

        if(len(list_contour) == 0):
            print("No contour found")
            continue
            
        if(best_contour is None):
            #get max of list_contour
            best_contour = max(list_contour, key=lambda x: contourArea(x))

        #----------------------------------------------------------------------------------------------------------

        (x,y,w,h) = boundingRect(best_contour)
        rectangle(raw_image, (x,y), (x+w,y+h), (0,255,0), 2)
        drawContours(raw_image, [best_contour],  -1, (255,0,0), 2)
        waitKey()
        raw_image = raw_image[y:y + h, x:x + w]
        img = imread(f)
        img = cvtColor(img, COLOR_BGR2RGB)   
        imwrite(dest+"/"+basename(realpath(f)).replace('.ppm', '.jpeg'), raw_image)

    return names

In [ ]:
from os.path import exists
from os import system, makedirs

# from Visu import show_images

path = "eval_kaggle2"
dest = "result_croppped_eval_2"

if exists(dest):
    system('rm -r dest')

if not exists(dest):
    makedirs(dest)

if not exists(path):
    makedirs(path)

names = process(path, dest)
#show_images(dest)

<font size=6  color= 'white' > <b> Phase 2 : Generating a CSV for kaggle </b> <br>

In [ ]:
from CSV import store_2

images, names = store_2('result_croppped_eval_2', resol)

In [ ]:
from csv import DictReader

# We will store the data in dict.csv in a dict 

data = {}

with open('dict.csv', 'r') as f:
    d_reader = DictReader(f, fieldnames=["num", "sign"])

    #get fieldnames from DictReader object and store in list
    for row in d_reader:
        data[row['num']] = row['sign']

In [ ]:
from matplotlib.pyplot import figure, title, imshow 
from numpy import uint8

# Here's a little test to visualise some results 
# We then print the image with num and the sign predicted as a title 

predictions = model_second.predict(images).argmax(axis=1) 

visu_pred = zeros(images.shape)

for i in range (images.shape[0]):
    for j in range (images.shape[3]) : 
        for k in range (images.shape[1]):
            for l in range (images.shape[2]): 
                visu_pred[i,k,l,j] = images[i,k,l,j] / 255

for i in range(20): 
    figure(figsize = (10,10))
    imshow(visu_pred[i])
    sign = data[str(predictions[i])]
    title(str(predictions[i]) + " : " + str(sign))

In [ ]:
# We generate the csv file for kaggle 

from CSV import write 

write(names, predictions, 'kaggle_2')